# Limitations de openEO

In [1]:
import openeo
from openeo.rest.connection import Connection

Pour utiliser OpenEO, on importe *openeo* et *openeo.rest.connection.Connection*. Ces modules permettent d'interagir avec l'API OpenEO et de communiquer avec les différents backends. On peut maintenant se connecter à un backend, effectuer des traitements sur les données, gérer les collections, sauvegarder les résultats sous différents formats, et bien plus encore. 

Les dépendances ci-dessous sont listées dans le *requirements.txt* du github d'[OpenEO](https://github.com/Open-EO/openeo-python-client), mais aucune version de bibliothèque n'est précisée. Cela peut entraîner des incompatibilités entre les différentes versions des bibliothèques, ce qui pourrait causer des erreurs lors de l'exécution de code. Les mises à jour des bibliothèques pourraient également rendre le code instable ou difficile à reproduire.

- numpy
- pandas
- matplotlib
- matplotlib-scalebar
- GDAL

Ici nous utilisons le backend OpenEO Copernicus Data Space Ecosystem, qui fournit des collections de données Sentinel et d'autres jeux de données issus de Copernicus. 

In [7]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu/")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

### Gestion des erreurs et exceptions

Les messages d’erreur retournés par les backends OpenEO peuvent être peu explicites, ce qui complique la compréhension et la résolution des problèmes. En effet, OpenEO renvoie des codes d'erreurs standardisés tels que 400 (Bad Request) ou 500 (Internal Server Error). Cependant, ces codes ne fournissent pas toujours suffisamment d'informations pour diagnostiquer le problème de manière efficace.

##### Exemples d'erreurs d'OpenEO

On utilise pour nos exemples un datacube de données Sentinel-2.

In [8]:
s2_timeseries = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=("2023-05-01", "2023-06-16"),
    spatial_extent={
        "west":  2.213649,
        "south": 43.450702,
        "east": 2.251248,
        "north":  43.472631,
        "crs": "EPSG:4326",
    },
    bands=["B02", "B03", "B04", "B08", "SCL"],
    max_cloud_cover=50,
);

Imaginons maintenant que l'utilisateur se trompe dans l'utilisation de la fonction *reduce_dimension()* et donne un mauvais *reducer*. Dans ce cas, aucune erreur ne se produit lors de la création du workflow :

In [9]:
s2_red = s2_timeseries.reduce_dimension(dimension="t", reducer="bands")

Cependant, lorsqu'on tente de sauvegarder le datacube avec la méthode *.download()*, l'erreur suivante apparaît :

In [10]:
s2_red.download("output/s2_red.nc")

OpenEoApiError: [400] BadRequest: java.lang.IllegalArgumentException: Unsupported operation: bands (arguments: [data, context]) (ref: r-241226a6d61d4baabfbf2eb45b28dec2)

Cette erreur indique que l'opération *bands* n'est pas supportée. Cependant, celle-ci ne permet pas d'identifier le traitement qui est la cause de l'erreur. Le message d'erreur devrait spécifier que le problème provient de la fonction reduce_dimension() et de l'argument reducer incorrect.

### Identification des librairies déployées sur un backend 

Pour identifier les librairies déployées sur un backend il est possible d'utiliser des fonctions définies par l'utilisateur (UDF). Elles permettent d'exécuter du code Python personnalisé directement sur le backend. 

On peut par exemple utiliser la librairie *importlib.metadata*, qui permet d'interroger les métadonnées des packages Python installés.

##### Exemple d'utilisation de UDF

Dans cet exemple, on re-scale notre datacube en utilisant une UDF. L'opération *.apply()* exécute l'UDF sur le datacube s2_timeseries, et on télécharge ensuite le résultat dans un fichier au format NetCDF.

In [ ]:
udf = openeo.UDF("""
import xarray
def apply_datacube(cube: xarray.DataArray, context: dict) -> xarray.DataArray:
    cube.values = 0.0005 * cube.values
    return cube
""")

s2_rescaled = s2_timeseries.apply(process=udf)

s2_rescaled.download("s2_rescaled.nc")